In [1]:
from collections import deque

In [2]:
def main():
    with open("input.txt") as f:
        grid = [line.rstrip('\n') for line in f]

    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    # Find S and E
    start = None
    end = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start = (r,c)
            elif grid[r][c] == 'E':
                end = (r,c)

    def is_track(r,c):
        return grid[r][c] in ('.','S','E')

    # Normal BFS neighbors: only track
    def neighbors(r,c):
        for dr,dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr,nc = r+dr,c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if is_track(nr,nc):
                    yield nr,nc

    def bfs(start):
        dist = [[None]*cols for _ in range(rows)]
        dist[start[0]][start[1]] = 0
        q = deque([start])
        while q:
            r,c = q.popleft()
            d = dist[r][c]
            for nr,nc in neighbors(r,c):
                if dist[nr][nc] is None:
                    dist[nr][nc] = d+1
                    q.append((nr,nc))
        return dist

    distFromStart = bfs(start)
    distToEnd = bfs(end)

    if distFromStart[end[0]][end[1]] is None:
        # No path to E even without cheating
        # Then no cheats matter (can't improve on impossible)
        print(0)  # Part 1
        print(0)  # Part 2
        return

    D_noCheat = distFromStart[end[0]][end[1]]

    # Collect all track cells
    track_cells = []
    for r in range(rows):
        for c in range(cols):
            if is_track(r,c):
                # Only consider A if it's reachable from S and E (otherwise no meaningful cheat)
                if distFromStart[r][c] is not None and distToEnd[r][c] is not None:
                    track_cells.append((r,c))

    cheat_savings_L2 = {}
    cheat_savings_L20 = {}

    directions = [(1,0),(-1,0),(0,1),(0,-1)]

    # Precompute a set of track cells for quick checking
    track_set = set(track_cells)

    def in_bounds(r,c):
        return 0 <= r < rows and 0 <= c < cols

    for (Ar,Ac) in track_cells:
        distA = distFromStart[Ar][Ac]
        # BFS ignoring walls up to 20 steps
        limit = 20
        dist_ignore = [[None]*cols for _ in range(rows)]
        dist_ignore[Ar][Ac] = 0
        q = deque([(Ar,Ac)])
        while q:
            r,c = q.popleft()
            d = dist_ignore[r][c]
            if d == limit:
                continue  # Can't go further than 20 steps
            for dr,dc in directions:
                nr,nc = r+dr,c+dc
                if in_bounds(nr,nc):
                    if dist_ignore[nr][nc] is None:
                        # Move ignoring walls
                        dist_ignore[nr][nc] = d+1
                        q.append((nr,nc))

        # Now dist_ignore gives the shortest ignoring-walls distance from A to any cell
        # We need to consider only track endpoints B
        for (Br,Bc) in track_cells:
            if (Br,Bc) == (Ar,Ac):
                continue
            d_ignore = dist_ignore[Br][Bc]
            if d_ignore is not None and d_ignore > 0:
                # d_ignore ≤ 20 means a valid cheat under the new rules
                distB = distToEnd[Br][Bc]
                D_cheat = distA + d_ignore + distB
                saving = D_noCheat - D_cheat
                if saving > 0:
                    # Update part two (L=20)
                    old = cheat_savings_L20.get(((Ar,Ac),(Br,Bc)), 0)
                    if saving > old:
                        cheat_savings_L20[((Ar,Ac),(Br,Bc))] = saving

                    # If d_ignore ≤ 2, also update part one (L=2)
                    if d_ignore <= 2:
                        old2 = cheat_savings_L2.get(((Ar,Ac),(Br,Bc)),0)
                        if saving > old2:
                            cheat_savings_L2[((Ar,Ac),(Br,Bc))] = saving

    # Count how many cheats have saving ≥ 100 for part one and part two
    count_part1 = sum(1 for s in cheat_savings_L2.values() if s >= 100)
    count_part2 = sum(1 for s in cheat_savings_L20.values() if s >= 100)

    print(count_part1)  # Part 1
    print(count_part2)  # Part 2

if __name__ == "__main__":
    main()

1387
1015092
